# Imports

In [1]:
import os
import re
import sys
import json
import geopandas as gpd
import xarray as xr
import numpy as np
import time

In [2]:
# go up until we are in the project base directory
base_dir = os.getcwd()
while base_dir.split('/')[-1] != 'provide':
    base_dir = os.path.normpath(os.path.join(base_dir, '..'))

# add paths for tools and data
things_to_add = ['general_tools', 'aggregation_tools', 'general_data_for_aggregation']
for thing in things_to_add:
    sys.path.append(os.path.join(base_dir, thing))

from general_tools import check_if_notebook, mkdir
from oggm_result_filepath_and_realisations import scenarios_mesmer
from aggregation_plots import plot_map, plot_timeseries, plot_unavoidable_risk

In [3]:
# Use this to conditionally execute tests/debugging
if check_if_notebook():
    is_notebook = True
else:
    is_notebook = False

# define inputs

In [4]:
resolution_dir = 'total_data'
input_folder = 'aggregated_data'
output_folder = 'aggregated_result_plots'

In [5]:
preprocess_region_dict_outpath = os.path.join(base_dir, 'basins', resolution_dir)

with open(os.path.join(preprocess_region_dict_outpath, "preprocessed_region_grids.json"), 'r') as f:
    region_structure_dict = json.load(f)

In [6]:
regions_data_dir = os.path.join(base_dir, 'basins', 'data')
regions_file = 'glacier_basins.shp'
gdf_regions = gpd.read_file(os.path.join(regions_data_dir, regions_file))
name_col_regions = 'full_name'

gdf_regions[name_col_regions] = [re.sub(r'\[.*?\]|\(.*?\)', '', name).strip().lower().replace(' ', '_')
                                 for name in gdf_regions['RIVER_BASI'].values]

# plot for all regions and scenarios

In [7]:
def plot_scenario_results_region(region_name, scenario, input_folder, output_folder,
                                 add_map_plot=True, resolution=None,
                                ):
    plot_output_folder = os.path.join(output_folder, region_name)
    mkdir(plot_output_folder)
    region = gdf_regions[gdf_regions[name_col_regions] == region_name]

    if add_map_plot:
        plot_map(region, region_name, scenario, input_folder,
                 resolution=resolution,
                 figsize=(12, 12),
                 save_plot=plot_output_folder)

    plot_timeseries(region_name, scenario, input_folder,
                    figsize=(5, 9),
                    save_plot=plot_output_folder)

def plot_unavoidable_risk_for_all_scenarios(region_name, scenarios, input_folder,
                                            output_folder):
    plot_output_folder = os.path.join(output_folder, region_name)
    mkdir(plot_output_folder)

    plot_unavoidable_risk(region_name, scenarios, input_folder, figsize=(5, 15),
                          save_plot=plot_output_folder)

## testing in notebook

In [8]:
if is_notebook:
    test_output_folder = 'aggregated_result_plots_test'
    mkdir(test_output_folder)

    test_region = 'ISL'
    test_scenario = scenarios_mesmer[0]

    plot_scenario_results_region(test_region, test_scenario,
                                  input_folder, test_output_folder)

    plot_unavoidable_risk_for_all_scenarios(test_region, scenarios_mesmer,
                                            input_folder, test_output_folder)

FileNotFoundError: [Errno 2] No such file or directory: '/home/www/pschmitt/provide/aggregate_data/github/provide/basins/total_data/aggregated_data/ISL/ISL_CurPol_map.nc'

## code for cluster

In [9]:
if not is_notebook:
    # save results on cluster and copy at the end in run_slurm-file
    working_dir_cluster = os.environ.get('OGGM_WORKDIR', None)

    output_folder = os.path.join(working_dir_cluster,
                                 output_folder)
    mkdir(output_folder)

    start_time = time.time()
    for region in region_structure_dict:
        print(f'Start plotting {region}:')
        print(f'    unavoidable risk ({time.time() - start_time:.1f} s)')
        plot_unavoidable_risk_for_all_scenarios(region, scenarios_mesmer,
                                                input_folder, output_folder)
        for scenario in scenarios_mesmer:
            print(f'    {scenario} plots ({time.time() - start_time:.1f} s)')
            plot_scenario_results_region(region, scenario,
                                         input_folder, output_folder,
                                         add_map_plot=True,
                                         resolution=1,
                                        )

# count files of each region

In [21]:
if is_notebook:
    nr_files_ref = None
    for region in region_structure_dict:
        path_region = os.path.join(output_folder,
                                    region)
        nr_files_region = len([file for file in os.listdir(path_region)
                               if os.path.isfile(os.path.join(path_region,file))])
        if nr_files_ref is None:
            nr_files_ref = nr_files_region
        elif nr_files_ref != nr_files_region:
            print(f'!!!{region} {nr_files_region} files, reference {nr_files_ref}!!!')
        else:
            print(f'{region} {nr_files_region} files')

ob 22 files
brahmaputra 22 files
ganges 22 files
indus 22 files
irrawaddy 22 files
salween 22 files
kamchatka 22 files
mekong 22 files
yangtze 22 files
yellow_river 22 files
aral_sea 22 files
chuy 22 files
har_us_nuur 22 files
lake_balkhash 22 files
talas 22 files
tarim_he 22 files
uvs_nuur 22 files
ysyk-kol 22 files
amazon 22 files
chico 22 files
colorado 22 files
magdalena 22 files
negro 22 files
santa_cruz 22 files
aisen 22 files
azopardo 22 files
baker 22 files
biobio 22 files
cisnes 22 files
copiapo 22 files
huasco 22 files
majes 22 files
ocona 22 files
palena 22 files
pascua 22 files
puelo 22 files
rapel 22 files
santa 22 files
serrano 22 files
valdivia 22 files
yelcho 22 files
titicaca 22 files
colville 22 files
mackenzie 22 files
nelson 22 files
alsek 22 files
columbia 22 files
copper 22 files
fraser 22 files
kuskokwim 22 files
nass 22 files
nushagak 22 files
skagit 22 files
skeena 22 files
stikine 22 files
susitna 22 files
taku 22 files
yukon 22 files
clutha 22 files
jokulsa_a

# check output files for consistancy

In [25]:
if is_notebook:
    # same ref values for each scenario?
    for region in region_structure_dict:
        print(f'Checking ref values {region}')
        ref_volume = None
        ref_area = None
        ref_runoff = None
        for scenario in scenarios_mesmer:
            with xr.open_dataset(
                os.path.join(input_folder,
                             region,
                             f'{region}_{scenario}_timeseries.nc')) as ds_time:
                if ref_volume is None:
                    ref_volume = ds_time.volume.reference_2020_km3
                else:
                    if not np.isclose(ds_time.volume.reference_2020_km3,
                                      ref_volume,
                                      #rtol=0.01,
                                      #atol=30
                                     ):
                        print(f'{region}/{scenario}: volume NOT close to reference '
                              f'(given {ds_time.volume.reference_2020_km3:.1f}, '
                              f'reference {ref_volume:.1f})')

                if ref_area is None:
                    ref_area = ds_time.area.reference_2020_km2
                else:
                    if not np.isclose(ds_time.area.reference_2020_km2,
                                      ref_area,
                                      #rtol=0.01,
                                      #atol=80
                                     ):
                        print(f'{region}/{scenario}: area NOT close to reference '
                              f'(given {ds_time.area.reference_2020_km2:.1f}, '
                              f'reference {ref_area:.1f})')

                if ref_runoff is None:
                    ref_runoff = ds_time.runoff.reference_2000_2019_Mt_per_yer
                else:
                    if not np.isclose(ds_time.runoff.reference_2000_2019_Mt_per_yer,
                                      ref_runoff,
                                      #rtol=0.01,
                                      #atol=80
                                     ):
                        print(f'{region}/{scenario}: runoff NOT close to reference '
                              f'(given {ds_time.runoff.reference_2000_2019_Mt_per_yer:.1f}, '
                              f'reference {ref_runoff:.1f})')

    # are map values 2020 add up to 100%
    for region in region_structure_dict:
        print(f'Checking map sum 2020 for {region}')
        for scenario in scenarios_mesmer:
            with xr.open_dataset(
                        os.path.join(input_folder,
                                     region,
                                     f'{region}_{scenario}_map.nc')) as ds_map:
                for var in ['volume', 'area']:
                    for quant in ds_map['quantile']:
                        map_sum = ds_map.loc[{'time': 2020, 'quantile':quant}][var].sum().values
                        if not np.isclose(map_sum, 100):
                            if np.isclose(map_sum, 0):
                                print(f'  {region} is 0 ({scenario}, {var}, {quant.values})')
                            else:
                                print(f'Map 2020 adds not up to 100, only {map_sum} '
                                      f'({region}, {scenario}, {var}, {quant.values})')

Checking ref values indigirka
Checking ref values ob
Checking ref values brahmaputra
Checking ref values ganges
Checking ref values indus
Checking ref values irrawaddy
Checking ref values salween
Checking ref values kamchatka
Checking ref values mekong
Checking ref values yangtze
Checking ref values yellow_river
Checking ref values aral_sea
Checking ref values chuy
Checking ref values har_us_nuur
Checking ref values lake_balkhash
Checking ref values talas
Checking ref values tarim_he
Checking ref values uvs_nuur
Checking ref values ysyk-kol
Checking ref values amazon
Checking ref values chico
Checking ref values colorado
Checking ref values magdalena
Checking ref values negro
Checking ref values santa_cruz
Checking ref values aisen
Checking ref values azopardo
Checking ref values baker
Checking ref values biobio
Checking ref values cisnes
Checking ref values copiapo
Checking ref values huasco
Checking ref values majes
Checking ref values ocona
Checking ref values palena
Checking ref va